In [2]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
print(os.listdir("./input"))

import lightgbm as lgb
from sklearn.model_selection import *
from sklearn.metrics import mean_squared_error, make_scorer
from scipy.stats import mode, skew, kurtosis, entropy
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
import seaborn as sns

import dask.dataframe as dd
from dask.multiprocessing import get

from tqdm import tqdm, tqdm_notebook
tqdm.pandas(tqdm_notebook)

['.DS_Store', 'feature_report.csv', 'sample_submission.csv', 'test.csv', 'test_leak_org.csv', 'train.csv', 'train_leak_org.csv']


In [3]:
train = pd.read_csv("./input/train.csv")
test = pd.read_csv("./input/test.csv")

transact_cols = [f for f in train.columns if f not in ["ID", "target"]]
y = np.log1p(train["target"]).values

In [4]:
test["target"] = train["target"].mean()

In [5]:
ordered_cols = ['ced6a7e91','9df4daa99','83c3779bf','edc84139a','f1e0ada11',
                '73687e512','aa164b93b','342e7eb03','cd24eae8a','8f3740670',
                '2b2a10857','a00adf70e','3a48a2cd2','a396ceeb9','9280f3d04',
                'fec5eaf1a','5b943716b','22ed6dba3','5547d6e11','e222309b0',
                '5d3b81ef8','1184df5c2','2288333b4','f39074b55','a8b721722',
                '13ee58af1','fb387ea33','4da206d28','ea4046b8d','ef30f6be5',
                'b85fa8b27','2155f5e16']

ordered_ids = ['a70090dc7','5b74875d4','ca6c4ea0d','242407fc6','7ff66b22b',
               'f43a48254','c9573f4ac','c14136677','08f544828','e77e32574',
               '61636fa41','4c0316d32','414871b56','2c518cd87','9eb93baef',
               'd63b2307d','db9ddeb12','7e45f5bd5','680ecfac8','814f58e7b',
               'eb850ef06','6edaf114d']

ordered_indexes = []
for oi in ordered_ids:
    ordered_indexes.append(train[train.ID == oi].index[0])

tmp = train.loc[ordered_indexes, ["ID","target"] + ordered_cols]
print('shape', tmp.shape)
tmp

shape (22, 34)


,ID,target,ced6a7e91,9df4daa99,83c3779bf,edc84139a,f1e0ada11,73687e512,aa164b93b,342e7eb03,...,2288333b4,f39074b55,a8b721722,13ee58af1,fb387ea33,4da206d28,ea4046b8d,ef30f6be5,b85fa8b27,2155f5e16
2848,a70090dc7,400000.00,900000.00,380000.00,1359000.00,320000.00,1432000.00,0.00,565142.86,532000.00,...,57714.28,346444.44,62000.00,0.00,0.00,514666.66,0.00,440500.00,132200.00,281000.00
1582,5b74875d4,707000.00,500000.00,900000.00,380000.00,1359000.00,320000.00,1432000.00,0.00,565142.86,...,332000.00,57714.28,346444.44,62000.00,0.00,0.00,514666.66,0.00,440500.00,132200.00
3509,ca6c4ea0d,547000.00,0.00,500000.00,900000.00,380000.00,1359000.00,320000.00,1432000.00,0.00,...,170000.00,332000.00,57714.28,346444.44,62000.00,0.00,0.00,514666.66,0.00,440500.00
626,242407fc6,11115200.00,831000.00,0.00,500000.00,900000.00,380000.00,1359000.00,320000.00,1432000.00,...,0.00,170000.00,332000.00,57714.28,346444.44,62000.00,0.00,0.00,514666.66,0.00
2191,7ff66b22b,2253333.34,819500.00,831000.00,0.00,500000.00,900000.00,380000.00,1359000.00,320000.00,...,380000.00,0.00,170000.00,332000.00,57714.28,346444.44,62000.00,0.00,0.00,514666.66
4261,f43a48254,25300000.00,1614909.10,819500.00,831000.00,0.00,500000.00,900000.00,380000.00,1359000.00,...,1771818.18,380000.00,0.00,170000.00,332000.00,57714.28,346444.44,62000.00,0.00,0.00
3493,c9573f4ac,819750.00,1234666.66,1614909.10,819500.00,831000.00,0.00,500000.00,900000.00,380000.00,...,100000.00,1771818.18,380000.00,0.00,170000.00,332000.00,57714.28,346444.44,62000.00,0.00
3340,c14136677,40000.00,2400000.00,1234666.66,1614909.10,819500.00,831000.00,0.00,500000.00,900000.00,...,653333.34,100000.00,1771818.18,380000.00,0.00,170000.00,332000.00,57714.28,346444.44,62000.00
156,08f544828,25100000.00,236857.14,2400000.00,1234666.66,1614909.10,819500.00,831000.00,0.00,500000.00,...,80000.00,653333.34,100000.00,1771818.18,380000.00,0.00,170000.00,332000.00,57714.28,346444.44
4040,e77e32574,22134000.00,371200.00,236857.14,2400000.00,1234666.66,1614909.10,819500.00,831000.00,0.00,...,60000.00,80000.00,653333.34,100000.00,1771818.18,380000.00,0.00,170000.00,332000.00,57714.28


In [6]:
cols = ['f190486d6', '58e2e02e6', 'eeb9cd3aa', '9fd594eec', '6eef030c1',
       '15ace8c9f', 'fb0f5dbfe', '58e056e12', '20aa07010', '024c577b9',
       'd6bb78916', 'b43a7cfd5', '58232a6fb', '1702b5bf0', '324921c7b', 
       '62e59a501', '2ec5b290f', '241f0f867', 'fb49e4212',  '66ace2992',
       'f74e8f13d', '5c6487af1', '963a49cdc', '26fc93eb7', '1931ccfdd', 
       '703885424', '70feb1494', '491b9ee45', '23310aa6f', 'e176a204a',
       '6619d81fc', '1db387535', 'fc99f9426', '91f701ba2',  '0572565c2',
       '190db8488',  'adb64ff71', 'c47340d97', 'c5a231d81', '0ff32eb98']

In [7]:
##extra_feats should be a list of feature lists


def get_log_pred(data, feats, extra_feats, lag=0):
    offset = lag + 2
    f1 = feats[:(offset * -1)]
    f2 = feats[offset:]
    for ef in extra_feats:
        f1 += ef[:(offset * -1)]
        f2 += ef[offset:]
    
    d1 = data[f1].apply(tuple, axis=1).to_frame().rename(columns={0: 'key'})
    d2 = data[f2].apply(tuple, axis=1).to_frame().rename(columns={0: 'key'})
    d2['pred'] = data[feats[offset-2]]
    d2 = d2[d2['pred'] != 0] # Keep?
    d3 = d2[~d2.duplicated(['key'], keep=False)]
    d4 = d1[~d1.duplicated(['key'], keep=False)]
    d5 = d4.merge(d3, how='inner', on='key')
    
    d = d1.merge(d5, how='left', on='key')
    return d.pred.fillna(0)


In [8]:
#def _get_leak(df, cols, lag=0):
#    d1 = df[cols[:-lag-2]].apply(tuple, axis=1).to_frame().rename(columns={0: 'key'})
#    d2 = df[cols[lag+2:]].apply(tuple, axis=1).to_frame().rename(columns={0: 'key'})
#    d2['pred'] = df[cols[lag]]
#    #d2 = d2[d2.pred != 0] ### to make output consistent with Hasan's function
#    d3 = d2[~d2.duplicated(['key'], keep=False)]
#    return d1.merge(d3, how='left', on='key').pred.fillna(0)

In [8]:
def compiled_leak_result():
    
    max_nlags = len(cols) - 2
    #train_leak = train[["ID", "target"] + cols]
    train_leak = train[["ID", "target"] + cols + ordered_cols]
    train_leak["compiled_leak"] = 0
    train_leak["nonzero_mean"] = train[transact_cols].apply(
        lambda x: np.expm1(np.log1p(x[x!=0]).mean()), axis=1
    )
    
    scores = []
    leaky_value_counts = []
    leaky_value_corrects = []
    leaky_cols = []
    
    for i in range(max_nlags):
        c = "leaked_target_"+str(i)
        
        print('Processing lag', i)
        #train_leak[c] = _get_leak(train_leak, cols, i)
        train_leak[c] = get_log_pred(train_leak, cols, [ordered_cols], i)
        
        leaky_cols.append(c)
        train_leak = train.join(
            train_leak.set_index("ID")[leaky_cols+["compiled_leak", "nonzero_mean"]], 
            on="ID", how="left"
        )[["ID", "target"] + cols + ordered_cols + leaky_cols+["compiled_leak", "nonzero_mean"]] ##changed 
        zeroleak = train_leak["compiled_leak"]==0
        train_leak.loc[zeroleak, "compiled_leak"] = train_leak.loc[zeroleak, c]
        leaky_value_counts.append(sum(train_leak["compiled_leak"] > 0))
        _correct_counts = sum(train_leak["compiled_leak"]==train_leak["target"])
        leaky_value_corrects.append(_correct_counts/leaky_value_counts[-1])
        print("Leak values found in train", leaky_value_counts[-1])
        print(
            "% of correct leaks values in train ", 
            leaky_value_corrects[-1]
        )
        tmp = train_leak.copy()
        tmp.loc[zeroleak, "compiled_leak"] = tmp.loc[zeroleak, "nonzero_mean"]
        scores.append(np.sqrt(mean_squared_error(y, np.log1p(tmp["compiled_leak"]).fillna(14.49))))
        print(
            'Score (filled with nonzero mean)', 
            scores[-1]
        )
    result = dict(
        score=scores, 
        leaky_count=leaky_value_counts,
        leaky_correct=leaky_value_corrects,
    )
    return train_leak, result

In [9]:
train_leak, result = compiled_leak_result()

Processing lag 0
Leak values found in train 1367
% of correct leaks values in train  1.0
Score (filled with nonzero mean) 1.5138333391635188
Processing lag 1
Leak values found in train 1969
% of correct leaks values in train  1.0
Score (filled with nonzero mean) 1.2872114366939749
Processing lag 2
Leak values found in train 2357
% of correct leaks values in train  1.0
Score (filled with nonzero mean) 1.165934282892103
Processing lag 3
Leak values found in train 2603
% of correct leaks values in train  1.0
Score (filled with nonzero mean) 1.0698781857562876
Processing lag 4
Leak values found in train 2770
% of correct leaks values in train  1.0
Score (filled with nonzero mean) 1.0146202572499015
Processing lag 5
Leak values found in train 2915
% of correct leaks values in train  1.0
Score (filled with nonzero mean) 0.9761430364936858
Processing lag 6
Leak values found in train 3034
% of correct leaks values in train  1.0
Score (filled with nonzero mean) 0.9304593328574149
Processing lag

In [10]:
best_score = np.min(result['score'])
best_lag = np.argmin(result['score'])
print('best_score', best_score, '\nbest_lag', best_lag)

best_score 0.6828850600038048 
best_lag 36


In [11]:
def rewrite_compiled_leak(leak_df, lag):
    leak_df["compiled_leak"] = 0
    for i in range(lag):
        c = "leaked_target_"+str(i)
        zeroleak = leak_df["compiled_leak"]==0
        leak_df.loc[zeroleak, "compiled_leak"] = leak_df.loc[zeroleak, c]
    return leak_df

In [12]:
leaky_cols = [c for c in train_leak.columns if 'leaked_target_' in c]

In [13]:
train_leak = rewrite_compiled_leak(train_leak, best_lag+1)
train_leak[['ID']+leaky_cols+['compiled_leak']].head()

,ID,leaked_target_0,leaked_target_1,leaked_target_2,leaked_target_3,leaked_target_4,leaked_target_5,leaked_target_6,leaked_target_7,leaked_target_8,...,leaked_target_29,leaked_target_30,leaked_target_31,leaked_target_32,leaked_target_33,leaked_target_34,leaked_target_35,leaked_target_36,leaked_target_37,compiled_leak
0,000d6aaf2,38000000.0,38000000.0,38000000.0,0.0,38000000.0,0.0,38000000.0,0.0,0.0,...,38000000.0,0.0,38000000.0,0.0,38000000.0,38000000.0,0.0,0.0,0.0,38000000.0
1,000fbd867,600000.0,0.0,0.0,0.0,0.0,0.0,600000.0,0.0,600000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,600000.0
2,0027d6b71,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0028cbf45,0.0,0.0,0.0,2000000.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2000000.0
4,002a68644,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
train_res = train_leak[["ID"]+leaky_cols+['compiled_leak']].replace(0.0, np.nan)
train_res.to_csv('train_leak_paradox.csv', index=False)

In [15]:
def compiled_leak_result_test(max_nlags):
    #test_leak = test[["ID", "target"] + cols]
    test_leak = test[["ID", "target"] + cols + ordered_cols]
    test_leak["compiled_leak"] = 0
    test_leak["nonzero_mean"] = test[transact_cols].apply(
        lambda x: np.expm1(np.log1p(x[x!=0]).mean()), axis=1
    )
    
    scores = []
    leaky_value_counts = []
    # leaky_value_corrects = []
    leaky_cols = []
    
    for i in range(max_nlags):
        c = "leaked_target_"+str(i)
        
        print('Processing lag', i)
        #test_leak[c] = _get_leak(test_leak, cols, i)
        
        test_leak[c] = get_log_pred(test_leak, cols, [ordered_cols], i)
        leaky_cols.append(c)
        test_leak = test.join(
            test_leak.set_index("ID")[leaky_cols+["compiled_leak", "nonzero_mean"]], 
            on="ID", how="left"
        )[["ID", "target"] + cols + ordered_cols + leaky_cols+["compiled_leak", "nonzero_mean"]]  #changed
        zeroleak = test_leak["compiled_leak"]==0
        test_leak.loc[zeroleak, "compiled_leak"] = test_leak.loc[zeroleak, c]
        leaky_value_counts.append(sum(test_leak["compiled_leak"] > 0))
        #_correct_counts = sum(train_leak["compiled_leak"]==train_leak["target"])
        #leaky_value_corrects.append(_correct_counts/leaky_value_counts[-1])
        print("Leak values found in test", leaky_value_counts[-1])
        #print(
        #    "% of correct leaks values in train ", 
        #    leaky_value_corrects[-1]
        #)
        #tmp = train_leak.copy()
        #tmp.loc[zeroleak, "compiled_leak"] = tmp.loc[zeroleak, "nonzero_mean"]
        #scores.append(np.sqrt(mean_squared_error(y, np.log1p(tmp["compiled_leak"]).fillna(14.49))))
        #print(
        #    'Score (filled with nonzero mean)', 
        #    scores[-1]
        #)
    result = dict(
        # score=scores, 
        leaky_count=leaky_value_counts,
        # leaky_correct=leaky_value_corrects,
    )
    return test_leak, result

In [16]:
test_leak, test_result = compiled_leak_result_test(max_nlags=38)

Processing lag 0
Leak values found in test 2950
Processing lag 1
Leak values found in test 4205
Processing lag 2
Leak values found in test 4939
Processing lag 3
Leak values found in test 5485
Processing lag 4
Leak values found in test 5903
Processing lag 5
Leak values found in test 6179
Processing lag 6
Leak values found in test 6379
Processing lag 7
Leak values found in test 6525
Processing lag 8
Leak values found in test 6679
Processing lag 9
Leak values found in test 6801
Processing lag 10
Leak values found in test 6909
Processing lag 11
Leak values found in test 6998
Processing lag 12
Leak values found in test 7072
Processing lag 13
Leak values found in test 7148
Processing lag 14
Leak values found in test 7202
Processing lag 15
Leak values found in test 7256
Processing lag 16
Leak values found in test 7303
Processing lag 17
Leak values found in test 7345
Processing lag 18
Leak values found in test 7371
Processing lag 19
Leak values found in test 7411
Processing lag 20
Leak values 

In [17]:
test_result = pd.DataFrame.from_dict(test_result, orient='columns')
test_result.T

,0,1,2,3,4,5,6,7,8,9,...,28,29,30,31,32,33,34,35,36,37
leaky_count,2950,4205,4939,5485,5903,6179,6379,6525,6679,6801,...,7635,7668,7714,7768,7832,7920,8023,8143,8255,8315


In [18]:
test_leak = rewrite_compiled_leak(test_leak, best_lag+1)
test_leak[['ID']+leaky_cols+['compiled_leak']].head()

,ID,leaked_target_0,leaked_target_1,leaked_target_2,leaked_target_3,leaked_target_4,leaked_target_5,leaked_target_6,leaked_target_7,leaked_target_8,...,leaked_target_29,leaked_target_30,leaked_target_31,leaked_target_32,leaked_target_33,leaked_target_34,leaked_target_35,leaked_target_36,leaked_target_37,compiled_leak
0,000137c73,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,00021489f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0004d7953,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,5002000.0,0.0,0.0,0.0,0.0,5002000.0
3,00056a333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,00056d8eb,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
test_res = test_leak[["ID"]+leaky_cols+['compiled_leak']].replace(0.0, np.nan)
test_res.to_csv('test_leak_paradox.csv', index=False)

In [20]:
test_leak.loc[test_leak["compiled_leak"]==0, "compiled_leak"] = test_leak.loc[test_leak["compiled_leak"]==0, "nonzero_mean"]

In [22]:
sub = test[["ID"]]
sub["target"] = test_leak["compiled_leak"]
sub.to_csv(f"baseline_sub_lag_{best_lag}.csv", index=False)
print(f"baseline_sub_lag_{best_lag}.csv saved")

baseline_sub_lag_29.csv saved


In [23]:
sub.head()

,ID,target
0,000137c73,2.209645e+06
1,00021489f,1.099644e+06
2,0004d7953,1.276252e+06
3,00056a333,7.871320e+06
4,00056d8eb,2.868883e+06
